In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Masking, Concatenate, GRU, Dense, Reshape, Conv1D
from model import CNN_with_mask

In [2]:
rally_num = rally_set.shape[0]
rally_size = rally_set.shape[1]
shot_size = 3
feature_dim = (rally_set.shape[-1], rally_space_set.shape[-1], rally_action_set.shape[-1], rally_result_set.shape[-1])
space_embed_size = 8
action_embed_size = 8
shot_embed_size = 16

NameError: name 'rally_set' is not defined

In [21]:
def model(feature_dim, space_embed_size, action_embed_size, shot_embed_size):
    '''
    framework: 
    1. 對 space, action 做 embeding, (input, output) = (feature_dim, embed_size)
    2. concat space, action, others 成一個 embedded vector for each atk, (input) =  ([feature_dim, embed_size, embed_size])
    3. 先做 embedding
    4. CNN, filters = shot_embed_size
    5. GRU
    '''
    # each input: 三個維度, rally shot feature
    input_others = keras.Input(shape=(rally_size, shot_size, feature_dim[0]))
    input_space = keras.Input(shape=(rally_size, shot_size, feature_dim[1]))
    input_action = keras.Input(shape=(rally_size, shot_size, feature_dim[2]))

    # space & action 先做 embedding, 再和 others concat
    embed_space_layer = Embedding(input_dim=feature_dim[1], output_dim=space_embed_size, mask_zero=True, name='Space_Embedding')
    embed_action_layer = Embedding(input_dim=feature_dim[2], output_dim=action_embed_size, mask_zero=True, name='Action_Embedding')
    masking_layer = Masking(mask_value=0)   # for input_others (還沒有經過mask)
    concat_layer = Concatenate(name='Input_Concat')

    embed_shot_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=3, name='Shot_Embedding')

    cnn_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=1, name='CNN_Layer')
    gru_layer = GRU(units=16, return_sequences=True, name='GRU_Layer')
    dense_layer = Dense(units=3, activation='softmax')

    # forward
    inputs = [input_others, input_space, input_action]

    embed_space = tf.squeeze(embed_space_layer(input_space), axis=3)
    embed_action = tf.squeeze(embed_action_layer(input_action), axis=3)
    masked_others = masking_layer(tf.cast(input_others, tf.float32))
    embed_input = concat_layer([masked_others, embed_space, embed_action])
    embed_shot = tf.squeeze(embed_shot_layer(embed_input), axis=2)

    cnn_output = cnn_layer(embed_shot)
    gru_output = gru_layer(cnn_output)
    output = dense_layer(gru_output)



    model = keras.Model(inputs=inputs, outputs=output, name='Classification')
    return model

In [22]:
rally_size = 6
shot_size = 3

In [23]:
model = model((16, 1, 1), 4, 8, 16)
model.summary()

Model: "Classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 6, 3, 16)]   0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 6, 3, 1)]    0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           [(None, 6, 3, 1)]    0                                            
__________________________________________________________________________________________________
tf.cast_5 (TFOpLambda)          (None, 6, 3, 16)     0           input_19[0][0]                   
_____________________________________________________________________________________